In [55]:
from pymongo import MongoClient
import requests
import pandas as pd
import os
from dotenv import load_dotenv
import json
import folium
import src.cleaning_functions as fn
import src.google_api_functions as gf
import src.mongo_query_functions as mf
load_dotenv()

True

In [56]:

dbName = "companies"
mongodbURL = f"mongodb://localhost/{dbName}"
print(mongodbURL)
client = MongoClient(mongodbURL, connectTimeoutMS=10000,serverSelectionTimeoutMS=10000)
db = client.get_database()

mongodb://localhost/companies


# Filtering for offices in England

In [57]:
#Find the all the different country codes to choose the one for england
query={'offices': { '$exists': 'true', '$not': {'$size': 0} }}
web_offices = list(db.companies.find(query,{"offices":1,"name":1}))

countries=set()
for i,e in enumerate(web_offices):
    countries.add(e['offices'][0]['country_code'])
#countries #GBR is the code for Great Britain

In [58]:
#Start with filtering only for companies with offices in England
query={'offices':{'$elemMatch':{'country_code':'GBR'}}}
project = {"name":1, "offices":1,'total_money_raised':1,'founded_year':1, 'category_code':1}
gb_offices = list(db.companies.find(query,project))

In [59]:
office_df = pd.DataFrame(gb_offices).explode('offices')

In [60]:
print(office_df.shape)
office_df.head(3)

(2574, 6)


,_id,name,category_code,founded_year,total_money_raised,offices
0,52cdef7c4bab8bd675297da0,Babelgum,games_video,2007.0,$13.2M,"{'description': '', 'address1': '', 'address2'..."
1,52cdef7c4bab8bd675297da6,SpinVox,messaging,NaN,$106M,"{'description': 'Corporate Headquarters', 'add..."
2,52cdef7c4bab8bd675297da8,OpenX,advertising,2008.0,$75.5M,"{'description': 'Headquarters', 'address1': '8..."


In [61]:
#Extract get geoJson format from column offices with the extract_loc function
offices = office_df.apply(fn.extract_loc,axis=1, result_type="expand")
offices.head(3)

,0,1,2
0,GBR,London,"{'type': 'Point', 'coordinates': [-6.267494, 5..."
1,GBR,Buckinghamshire,None
2,USA,Pasadena,"{'type': 'Point', 'coordinates': [-118.1327468..."


In [71]:
#Append it to office_df
offices_loc = pd.concat([office_df, offices], axis=1)
offices_loc.columns = ['__id', 'name', 'category_code','founded_year',  'raised', 'offices','country', 'city', 'location']

#Only offices in england
offices_loc = offices_loc[offices_loc.country=='GBR']

#Only offices with a valid lat long
offices_loc = offices_loc[~offices_loc.location.isnull()]

#Explore the result
print(offices_loc.shape)
print(offices_loc.dtypes)
offices_loc.head(3)

(611, 9)
__id              object
name              object
category_code     object
founded_year     float64
raised            object
offices           object
country           object
city              object
location          object
dtype: object


,__id,name,category_code,founded_year,raised,offices,country,city,location
0,52cdef7c4bab8bd675297da0,Babelgum,games_video,2007.0,$13.2M,"{'description': '', 'address1': '', 'address2'...",GBR,London,"{'type': 'Point', 'coordinates': [-6.267494, 5..."
4,52cdef7c4bab8bd675297dcc,AllPeers,web,2004.0,$0,"{'description': None, 'address1': None, 'addre...",GBR,Oxford,"{'type': 'Point', 'coordinates': [-1.255824, 5..."
5,52cdef7c4bab8bd675297de9,Zamzar,web,2006.0,$0,"{'description': '', 'address1': '', 'address2'...",GBR,Southampton,"{'type': 'Point', 'coordinates': [-1.3610845, ..."


In [72]:
#Formatting the raised money column with the clean_raised function
offices_loc['raised'] = offices_loc.raised.apply(fn.clean_raised)

In [73]:
offices_loc=offices_loc[['name', 'founded_year', 'category_code', 'raised', 'city', 'location']].reset_index()
offices_loc['location'][4]

{'type': 'Point', 'coordinates': [-2.250166, 53.473606]}

In [74]:
#Export to json:
offices_loc.to_json("OUTPUT/offices_loc.json",orient="records")

In [75]:
#Incorporating new collection to database and create geoloc index
!mongoimport --db companies --collection offices_loc_england --drop --jsonArray OUTPUT/offices_loc.json


2020-04-20T12:00:33.456+0200	connected to: mongodb://localhost/
2020-04-20T12:00:33.457+0200	dropping: companies.offices_loc_england
2020-04-20T12:00:33.816+0200	611 document(s) imported successfully. 0 document(s) failed to import.


In [77]:
db.list_collection_names()
#Create a GeoJson index for this collection
db.offices_loc_england.create_index([("location", '2dsphere')])

'location_2dsphere'

In [78]:
#Make sure that I assigned a correct GeoJson index in Mongo compass
db.offices_loc_england.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)], 'ns': 'companies.offices_loc_england'},
 'location_2dsphere': {'v': 2,
  'key': [('location', '2dsphere')],
  'ns': 'companies.offices_loc_england',
  '2dsphereIndexVersion': 3}}

## Filtering to accomodate developers: top notch companies (raised 1 mill dollars) nearby

In [79]:
offices = list(db.offices_loc_england.find({}))

In [81]:
len(offices)

611

In [82]:
#Check wich categories are available to filter out those I am not interested in.
p = list(db.companies.find({},{'category_code':1}))
category = set()
for e in p:
    category.add(e['category_code'])
#category 

In [83]:
#Filter for offices with tech companies that have raised more than 1 mill within 5km range
new_offices = mf.near_filter(db.offices_loc_england, offices,mf.near_success_offices, 'pro_nearby')
len(new_offices)

404

## Second filter: travelling for executives


In [86]:
#Get apiKey for google:
apiKey = os.getenv("GOOGLE_API2")

In [91]:
##Offices that are within 20 kilometers from an international airport
filtered_lst=[]
for of in new_offices:
    n_airport = len(gf.get_google(gf.get_latlong(of), radius=20000, typ ='airport',apiKey=apiKey, keyw='international'))
    if n_airport > 0:
        of['airp_nearby'] = n_airport 
        filtered_lst.append(of)

In [92]:
#Luckily most of our candidates are in London and Manchester, where travelling is easy
len(filtered_lst)

378

## Kindergartens and nurseries and primary schools

In [ ]:
##Offices that are within 1 kilometers from a nursery or kindergarten. 
#This time we will be a bit more demanding and make the name include one of a series of words:
kinder_lst = gf.school_kinder_filter(kinder_lst, ['kindergarten', 'nursery', 'primary'],'kinder', 
                                  radius=1000, typ='kindergarten', keyw='kindergarten', apiKey=apiKey)

In [98]:
len(kinder_lst)

199

In [112]:
#For those with kids a bit older, let's make sure that they also have a primary school nearby
school_lst = gf.school_kinder_filter(kinder_lst, ['school', 'primary'],'school', 
                                  radius=1000, typ='school', keyw='primary', apiKey=apiKey)

In [121]:
#It looks that kindergartens are way more limiting than primary schools
len(school_lst)

198

## Starbucks


In [114]:
#The API from starbucks does not allow any email domain to register, 
# therefore, I will use actual store lat long from a kaggle dataset

starbuck = pd.read_csv('INPUT/starbucks_stores.csv')
#filtering to stores in england
starbuck = starbuck[starbuck.Country=='GB']

In [115]:
#Using the create_geojson function, I create a location geoJson in this df
starbuck['location'] = starbuck.apply(mf.create_geojson, axis=1)

In [116]:
#Drop the useless columns and export to a json
starbucks = starbuck[['Store Name', 'City', 'location']]
starbucks.to_json("OUTPUT/starbucks.json",orient="records")

In [117]:
!mongoimport --db companies --collection starbucks --drop --jsonArray OUTPUT/starbucks.json

2020-04-20T13:00:00.719+0200	connected to: mongodb://localhost/
2020-04-20T13:00:00.720+0200	dropping: companies.starbucks
2020-04-20T13:00:01.089+0200	901 document(s) imported successfully. 0 document(s) failed to import.


In [118]:
#CReate geoJson index
db.starbucks.create_index([("location", '2dsphere')])

'location_2dsphere'

In [126]:
db.starbucks.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)], 'ns': 'companies.starbucks'},
 'location_2dsphere': {'v': 2,
  'key': [('location', '2dsphere')],
  'ns': 'companies.starbucks',
  '2dsphereIndexVersion': 3}}

In [157]:
#Filter the offices to those that within 200m from an starbucks

starbucks_lst=[]
for of in school_lst:
    n_strbk = mf.near_starbuck(db.starbucks, of['location'], dist=200)
    if n_strbk > 0:
        of['starbucks'] = n_strbk 
        starbucks_lst.append(of)

/home/david/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [158]:
len(starbucks_lst)

22

## The vegan CEO

In [160]:
#Make sure the CEO can get her vegan lunch
ceo_lst = []
for of in starbucks_lst:
    n = len(gf.get_google(gf.get_latlong(of), radius=200, typ='restaurant', keyw='vegan', apiKey=apiKey))
    if n > 0:
        of['starbucks'] = n 
        ceo_lst.append(of)

In [161]:
len(ceo_lst)

20

## Friday disco party


In [165]:
#Places with night life within 300m
final = []
for of in ceo_lst:
    n = len(gf.get_google(gf.get_latlong(of), radius=300, typ='bar', keyw='night-club', apiKey=apiKey))
    if n > 0:
        of['disco'] = n 
        final.append(of)

In [166]:
len(final)

19

**We are left with a total of 19 candidates for the location of our new office**

In [168]:
#Now we can sort the values according our findings following the priorities
final_df = pd.DataFrame(final)
final_df=final_df.drop('_id', axis=1)
final_df = final_df.sort_values(by=['pro_nearby', 
                                    'airp_nearby', 'kinder',
                                    'school', 'starbucks','disco'], ascending=False).reset_index()
final_df.head()


,level_0,index,name,founded_year,category_code,raised,city,location,pro_nearby,airp_nearby,kinder,school,starbucks,disco
0,6,397,Big Property Ladder,1995.0,other,0.000,London.,"{'type': 'Point', 'coordinates': [-0.1392765, ...",37,7,1,6,7,19
1,10,785,Quick TV,2007.0,enterprise,3.613,London,"{'type': 'Point', 'coordinates': [-0.1392765, ...",37,7,1,6,7,19
2,15,1070,Eglue Business Technologies,2001.0,software,27.000,London,"{'type': 'Point', 'coordinates': [-0.1392765, ...",37,7,1,6,7,19
3,8,576,litl,2007.0,hardware,0.000,London,"{'type': 'Point', 'coordinates': [-0.139244, 5...",37,7,1,6,7,18
4,9,590,Box UK,1998.0,software,0.000,London,"{'type': 'Point', 'coordinates': [-0.1392447, ...",37,7,1,6,7,18


In [169]:
final_df.to_json("OUTPUT/final.json",orient="records")

## Visualizing

In [210]:
final_df.location[0]['coordinates']

[-0.1392765, 51.5115309]

In [219]:
m = folium.Map(location=[51.520779, -0.10759],
               zoom_start=13,
              tiles='OpenStreet Map'#'Stamen Terrain'
              )


tooltip = 'Click me!'

for e in final_df.iterrows():
    html = f"""
    <strong> Ranking : #{e[0]}</strong>
    <p>
    Pro_companies:{e[1][8]},
    airports:{e[1][9]},
    kinder:{e[1][10]},
    schools:{e[1][11]},
    starbucks:{e[1][12]},
    discos:{e[1][13]}
    </p>
    """
    long= e[1][7]['coordinates'][-1]
    lat=e[1][7]['coordinates'][0]
    if e[0]<5:
        color='green'
    elif e[0]<10: color='blue'
    else: color='red'
    folium.Marker([long, lat], 
                  popup=html, 
                  tooltip=tooltip, 
                  icon=folium.Icon(color=color)).add_to(m)

folium.CircleMarker(
    location=[51.5115309, -0.1392765],
    radius=50,
    popup='Optimal area',
    color='#3186cc',
    fill=True,
    fill_color='#3186cc'
).add_to(m)

#for i,e in enumerate(final_df['location']):
#    i = folium.GeoJson(
#        data=e,
#        tooltip=tooltip)
#
#    i.add_child(folium.Popup('outline Popup on GeoJSON'))
#    i.add_to(m)


m.add_child(folium.LatLngPopup())


In [221]:
m.save('OUTPUT/html_map.html')